In [1]:
# Cell 1: Clone STG-NF và AlphaPose
!git clone https://github.com/orhir/STG-NF.git /kaggle/working/STG-NF
!git clone https://github.com/MVIG-SJTU/AlphaPose.git /kaggle/working/AlphaPose

Cloning into '/kaggle/working/STG-NF'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 449 (delta 284), reused 423 (delta 265), pack-reused 0 (from 0)
Receiving objects: 100% (449/449), 827.77 KiB | 4.33 MiB/s, done.
Resolving deltas: 100% (284/284), done.
Cloning into '/kaggle/working/AlphaPose'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 2749 (delta 4), reused 0 (delta 0), pack-reused 2740 (from 2)
Receiving objects: 100% (2749/2749), 118.82 MiB | 39.53 MiB/s, done.
Resolving deltas: 100% (1379/1379), done.


In [2]:
%cd /kaggle/working/AlphaPose
!git clone https://github.com/YuliangXiu/PoseFlow.git

/kaggle/working/AlphaPose
Cloning into 'PoseFlow'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 165 (delta 33), reused 25 (delta 9), pack-reused 110 (from 1)
Receiving objects: 100% (165/165), 9.52 MiB | 38.99 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [3]:
%%capture
!pip install "numpy==1.26.4"
!pip install cython pyyaml yacs tqdm matplotlib cython-bbox
!python setup.py build develop

In [4]:
import json
p="/kaggle/input/shanghai-alphapose/alphapose_out/train/01_001_alphapose_results.json"
d=json.load(open(p))
print(type(d))
if isinstance(d, dict):
    print("dict keys:", list(d.keys())[:30])
elif isinstance(d, list):
    print("list len:", len(d))
    print("first item keys:", list(d[0].keys())[:30])

<class 'list'>
list len: 1247
first item keys: ['image_id', 'category_id', 'keypoints', 'score', 'box', 'idx']


In [5]:
%%capture
%cd /kaggle/working/AlphaPose

!mkdir -p detector/yolo/data
!wget -O detector/yolo/data/yolov3-spp.weights "https://pjreddie.com/media/files/yolov3-spp.weights"

# Tạo thư mục chứa weights (nếu chưa có)
!mkdir -p pretrained_models
# Copy file .pth bạn đã upload vào Kaggle Input
!cp /kaggle/input/pretrained-models/fast_421_res152_256x192.pth \
    pretrained_models/fast_421_res152_256x192.pth

# TRACKING

In [6]:
import cv2
from pathlib import Path

# =======================
# CẤU HÌNH
# =======================
VIDEO_DIR = Path("/kaggle/input/shanghaitech/ShangHaiTech/training/videos")
OUT_ROOT  = Path("/kaggle/working/frames")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 100               # số video mỗi batch
RUN_BATCH = [4]               # <<< CHỈ CHẠY BATCH NÀY (batch bắt đầu từ 1)
# Ví dụ:
# RUN_BATCH = [1]          → chỉ chạy batch 1
# RUN_BATCH = [2, 4]       → chạy 2 batch
# RUN_BATCH = list(range(1,6)) → chạy tất cả batch

# =======================
# LẤY TOÀN BỘ VIDEO
# =======================
videos = sorted([v for v in VIDEO_DIR.iterdir() if v.suffix.lower() in [".mp4", ".avi", ".mov"]])
total_videos = len(videos)

print(f"Found {total_videos} videos.")

# =======================
# CHIA THÀNH BATCH
# =======================
batches = [
    videos[i:i+BATCH_SIZE]
    for i in range(0, total_videos, BATCH_SIZE)
]

print(f"Tổng số batch: {len(batches)} (mỗi batch {BATCH_SIZE} video)")
print(f"Batch được chọn để chạy: {RUN_BATCH}")

# =======================
# XỬ LÝ TỪNG BATCH
# =======================
for batch_idx, batch in enumerate(batches, start=1):

    if batch_idx not in RUN_BATCH:
        continue  # bỏ qua batch này

    print(f"\n==============================")
    print(f"      CHẠY BATCH {batch_idx}/{len(batches)}")
    print(f"      Số video: {len(batch)}")
    print(f"==============================")

    for vid in batch:
        vid_name = vid.stem
        out_dir = OUT_ROOT / vid_name
        out_dir.mkdir(parents=True, exist_ok=True)

        print(f"\n=== Extracting: {vid_name} ===")

        cap = cv2.VideoCapture(str(vid))
        frame_id = 0

        # --- trích frame ---
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_path = out_dir / f"{frame_id:06d}.jpg"
            cv2.imwrite(str(frame_path), frame)
            frame_id += 1

        cap.release()
        print(f"Extracted {frame_id} frames.")

        # --- đổi tên về <id>.jpg ---
        print("Renaming frames...")
        for img in sorted(out_dir.glob("*.jpg")):
            old = img.stem
            if old.isdigit():
                new = f"{int(old)}.jpg"
                img.rename(img.with_name(new))

        print(f"Done video {vid_name}.")

print("\n=== DONE: tất cả batch yêu cầu đã chạy xong ===")

Found 330 videos.
Tổng số batch: 4 (mỗi batch 100 video)
Batch được chọn để chạy: [4]

      CHẠY BATCH 4/4
      Số video: 30

=== Extracting: 11_003 ===
Extracted 675 frames.
Renaming frames...
Done video 11_003.

=== Extracting: 11_004 ===
Extracted 250 frames.
Renaming frames...
Done video 11_004.

=== Extracting: 11_005 ===
Extracted 1100 frames.
Renaming frames...
Done video 11_005.

=== Extracting: 11_006 ===
Extracted 550 frames.
Renaming frames...
Done video 11_006.

=== Extracting: 11_007 ===
Extracted 475 frames.
Renaming frames...
Done video 11_007.

=== Extracting: 11_008 ===
Extracted 625 frames.
Renaming frames...
Done video 11_008.

=== Extracting: 11_009 ===
Extracted 1800 frames.
Renaming frames...
Done video 11_009.

=== Extracting: 11_010 ===
Extracted 525 frames.
Renaming frames...
Done video 11_010.

=== Extracting: 12_001 ===
Extracted 425 frames.
Renaming frames...
Done video 12_001.

=== Extracting: 12_002 ===
Extracted 750 frames.
Renaming frames...
Done video

In [7]:
pip -q install torchreid opencv-python scipy

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/munkres-1.1.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/timm-0.1.20-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 103.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 111.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following

In [8]:
!pip -q uninstall -y tensorboard protobuf
!pip -q install "protobuf==3.20.3" "tensorboard==2.13.0"

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/munkres-1.1.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/timm-0.1.20-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opente

In [9]:
# ============================================================
# Batch reID tracking for AlphaPose JSONs -> *_tracked_person.json
# Fixes:
#  - Map json filename like "01_001_alphapose_results.json" -> frame folder "01_001"
#  - Batch processing: 50 videos per batch
#  - Debug stats to diagnose 0 tracks
# ============================================================

import os, json, glob, cv2
import numpy as np
from collections import defaultdict
from scipy.optimize import linear_sum_assignment

import torch
from torchvision import transforms
from torchreid import models

# ----------------------------
# USER CONFIG
# ----------------------------
ALPHAPOSE_DIR = "/kaggle/input/shanghai-alphapose/alphapose_out/train"
FRAME_ROOT    = "/kaggle/working/frames"
OUT_DIR       = "/kaggle/working/tracked_json"
os.makedirs(OUT_DIR, exist_ok=True)

# Batch
BATCH_SIZE = 100
BATCH_INDEX = 3  # 0 = first 50, 1 = next 50, ...

# Tracking params
COS_THR   = 0.30
IOU_THR   = 0.05
ALPHA     = 0.85
MAX_MISS  = 30
MIN_HITS  = 2
EMA       = 0.90

# Export filters
MIN_TRACK_LEN = 24     # paper T=24; set 1 to debug quickly if still 0
REQUIRE_CONFIRMED = True

# ----------------------------
# ReID model
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
reid_model = models.build_model(name="osnet_x1_0", num_classes=1000, pretrained=True).to(device).eval()

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 128)),  # H,W
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

@torch.inference_mode()
def extract_feat(img_bgr, box_xywh):
    x, y, w, h = map(float, box_xywh)
    x2, y2 = x + w, y + h
    H, W = img_bgr.shape[:2]
    x1, y1 = max(0, int(x)),  max(0, int(y))
    x2, y2 = min(W-1, int(x2)), min(H-1, int(y2))
    if x2 <= x1 or y2 <= y1:
        return None
    crop = img_bgr[y1:y2, x1:x2]
    if crop.size == 0:
        return None
    crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    x = preprocess(crop_rgb).unsqueeze(0).to(device)
    feat = reid_model(x)
    feat = torch.nn.functional.normalize(feat, dim=1)
    return feat.squeeze(0).detach().cpu().numpy()

def bbox_xywh_to_xyxy(b):
    x,y,w,h = map(float, b)
    return [x, y, x+w, y+h]

def iou_xyxy(a, b):
    ax1,ay1,ax2,ay2 = a
    bx1,by1,bx2,by2 = b
    ix1,iy1 = max(ax1,bx1), max(ay1,by1)
    ix2,iy2 = min(ax2,bx2), min(ay2,by2)
    iw,ih = max(0, ix2-ix1), max(0, iy2-iy1)
    inter = iw*ih
    area_a = max(0,ax2-ax1)*max(0,ay2-ay1)
    area_b = max(0,bx2-bx1)*max(0,by2-by1)
    return inter / (area_a + area_b - inter + 1e-6)

def cos_dist(f1, f2):
    return 1.0 - float(np.dot(f1, f2))

def parse_frame_id_from_image_id(image_id):
    s = str(image_id)
    if "/" in s:
        _, fname = s.split("/", 1)
        return os.path.splitext(fname)[0]
    base = os.path.splitext(s)[0]
    # if ends with _000123 -> take last
    if "_" in base:
        return base.split("_")[-1]
    return base

def load_frame(frame_folder_name, frame_id):
    folder = os.path.join(FRAME_ROOT, frame_folder_name)
    for ext in [".jpg", ".png", ".jpeg"]:
        p = os.path.join(folder, f"{frame_id}{ext}")
        if os.path.exists(p):
            img = cv2.imread(p)
            if img is not None:
                return img
    return None

class Track:
    def __init__(self, tid):
        self.id = tid
        self.feat = None
        self.bbox_xyxy = None
        self.miss = 0
        self.hits = 0
        self.confirmed = False
        self.frames = {}  # frame_id -> {"keypoints": [...], "scores": float}

def hungarian_match(tracks, dets):
    if len(tracks) == 0 or len(dets) == 0:
        return [], list(range(len(tracks))), list(range(len(dets)))

    C = np.full((len(tracks), len(dets)), 1e6, dtype=np.float32)

    for i, tr in enumerate(tracks):
        for j, dt in enumerate(dets):
            if tr.feat is None or dt["feat"] is None:
                continue
            d_app = cos_dist(tr.feat, dt["feat"])
            iou = iou_xyxy(tr.bbox_xyxy, dt["bbox_xyxy"])
            if d_app > COS_THR and iou < IOU_THR:
                continue
            d_iou = 1.0 - iou
            C[i, j] = ALPHA * d_app + (1.0 - ALPHA) * d_iou

    r, c = linear_sum_assignment(C)
    matches, used_t, used_d = [], set(), set()

    for rr, cc in zip(r, c):
        if C[rr, cc] >= 1e5:
            continue
        matches.append((rr, cc))
        used_t.add(rr)
        used_d.add(cc)

    un_t = [i for i in range(len(tracks)) if i not in used_t]
    un_d = [j for j in range(len(dets)) if j not in used_d]
    return matches, un_t, un_d

def safe_sort_frame_ids(frame_ids):
    try:
        return sorted(frame_ids, key=lambda x: int(str(x)))
    except Exception:
        return sorted(frame_ids, key=lambda x: str(x))

def frame_folder_from_json_name(video_name):
    # "01_001_alphapose_results" -> "01_001"
    if video_name.endswith("_alphapose_results"):
        return video_name.replace("_alphapose_results", "")
    return video_name

# ----------------------------
# MAIN
# ----------------------------
json_files_all = sorted(glob.glob(os.path.join(ALPHAPOSE_DIR, "*.json")))
print("Found alphapose json files:", len(json_files_all))

start = BATCH_INDEX * BATCH_SIZE
end = min(len(json_files_all), start + BATCH_SIZE)
json_files = json_files_all[start:end]
print(f"Running batch {BATCH_INDEX}: files [{start}:{end}] ({len(json_files)} videos)")

global_next_tid = 1

for json_path in json_files:
    video_name = os.path.splitext(os.path.basename(json_path))[0]
    frame_folder = frame_folder_from_json_name(video_name)

    frame_dir = os.path.join(FRAME_ROOT, frame_folder)
    if not os.path.isdir(frame_dir):
        print(f"\n=== {video_name} ===")
        print("SKIP: frame folder not found:", frame_dir)
        continue

    with open(json_path, "r") as f:
        det_list = json.load(f)

    by_frame = defaultdict(list)
    for det in det_list:
        frame_id = parse_frame_id_from_image_id(det.get("image_id"))
        by_frame[str(frame_id)].append(det)

    frame_ids = safe_sort_frame_ids(list(by_frame.keys()))
    if len(frame_ids) == 0:
        print(f"\n=== {video_name} ===")
        print("SKIP: no frames in json")
        continue

    active, finished = [], []

    frames_loaded = 0
    det_count = 0
    det_with_feat = 0

    for frame_id in frame_ids:
        img = load_frame(frame_folder, frame_id)
        if img is None:
            for tr in active:
                tr.miss += 1
            continue

        frames_loaded += 1
        dets_raw = by_frame[frame_id]

        dets = []
        for d in dets_raw:
            box = d.get("box", None)
            kps = d.get("keypoints", None)
            sc  = float(d.get("score", 0.0) or 0.0)
            if box is None or kps is None:
                continue

            det_count += 1
            feat = extract_feat(img, box)
            if feat is not None:
                det_with_feat += 1

            dets.append({
                "bbox_xyxy": bbox_xywh_to_xyxy(box),
                "feat": feat,
                "keypoints": kps,
                "score": sc
            })

        if len(active) == 0:
            for dt in dets:
                tr = Track(global_next_tid); global_next_tid += 1
                tr.bbox_xyxy = dt["bbox_xyxy"]
                tr.feat = dt["feat"]
                tr.hits = 1
                tr.confirmed = (tr.hits >= MIN_HITS)
                tr.frames[str(frame_id)] = {"keypoints": dt["keypoints"], "scores": dt["score"]}
                active.append(tr)
            continue

        matches, un_t, un_d = hungarian_match(active, dets)

        for ti, dj in matches:
            tr = active[ti]
            dt = dets[dj]
            tr.bbox_xyxy = dt["bbox_xyxy"]
            if tr.feat is None:
                tr.feat = dt["feat"]
            elif dt["feat"] is not None:
                tr.feat = EMA * tr.feat + (1.0 - EMA) * dt["feat"]
                tr.feat = tr.feat / (np.linalg.norm(tr.feat) + 1e-12)
            tr.miss = 0
            tr.hits += 1
            tr.confirmed = tr.confirmed or (tr.hits >= MIN_HITS)
            tr.frames[str(frame_id)] = {"keypoints": dt["keypoints"], "scores": dt["score"]}

        for ti in un_t:
            active[ti].miss += 1

        for dj in un_d:
            dt = dets[dj]
            tr = Track(global_next_tid); global_next_tid += 1
            tr.bbox_xyxy = dt["bbox_xyxy"]
            tr.feat = dt["feat"]
            tr.hits = 1
            tr.confirmed = (tr.hits >= MIN_HITS)
            tr.frames[str(frame_id)] = {"keypoints": dt["keypoints"], "scores": dt["score"]}
            active.append(tr)

        still = []
        for tr in active:
            if tr.miss > MAX_MISS:
                finished.append(tr)
            else:
                still.append(tr)
        active = still

    finished.extend(active)

    # Export
    out_all = {}
    out = {}
    for tr in finished:
        out_all[str(tr.id)] = tr.frames
        if REQUIRE_CONFIRMED and (not tr.confirmed):
            continue
        if len(tr.frames) < MIN_TRACK_LEN:
            continue
        out[str(tr.id)] = tr.frames

    out_path = os.path.join(OUT_DIR, f"{frame_folder}_alphapose_tracked_person.json")
    with open(out_path, "w") as f:
        json.dump(out, f)

    print(f"\n=== {video_name} (frame_folder={frame_folder}) ===")
    print("frames_loaded:", frames_loaded, "| det:", det_count, "| det_with_feat:", det_with_feat)
    print("tracks_total(before_filter):", len(out_all), "| tracks_saved(after_filter):", len(out))
    print("Saved:", out_path)

print("\nDONE. Output folder:", OUT_DIR)

/usr/local/lib/python3.11/dist-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
2026-01-30 11:28:10.107255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769772490.285075      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769772490.333105      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 85.5MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
Found alphapose json files: 329
Running batch 3: files [300:329] (29 videos)

=== 11_004_alphapose_results (frame_folder=11_004) ===
frames_loaded: 145 | det: 149 | det_with_feat: 149
tracks_total(before_filter): 3 | tracks_saved(after_filter): 1
Saved: /kaggle/working/tracked_json/11_004_alphapose_tracked_person.json

=== 11_005_alphapose_results (frame_folder=11_005) ===
frames_loaded: 434 | det: 830 | det_with_feat: 830
tracks_total(before_filter): 10 | tracks_saved(after_filter): 8
Saved: /kaggle/working/tracked_json/11_005_alphapose_tracked_person.json

=== 11_006_alphapose_results (frame_folder=11_006) ===
frames_loaded: 467 | det: 874 | det_with_feat: 874
tracks_total(before_filter): 5 | tracks_saved(after_filter): 5
Saved: /kaggle/working/tracked_json/11_006_alphapose_tracked_person.json

=== 11_007_alphapose_results (frame_folder=11_007) ===
frames_loaded: 404 | det: 6

In [10]:
import shutil

input_dir = "/kaggle/working/tracked_json"
output_zip = "/kaggle/working/tracked_json"

shutil.make_archive(output_zip, 'zip', input_dir)

print("Đã tạo file:", output_zip + ".zip")

Đã tạo file: /kaggle/working/tracked_json.zip
